In [7]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [2]:
#loading data

X = pd.read_csv('newdata.txt',header=None,sep=" ")
Y = pd.read_csv('newlabels.txt',header=None, sep=" ")
X=X.iloc[:,2:]
print(X.shape)
print(Y.shape)


(2464, 79)
(2464, 1)


In [3]:
# shuffling data

#idx = np.random.permutation(X.index)
#X = X.reindex(idx)
#Y = Y.reindex(idx)
#Y.head()

In [4]:
# spliiting training and  testing data
n=int(X.shape[0]*0.80)
length=X.shape[0]
X_train=X.loc[0:n,:]
X_test1=X.loc[n:,:]

Y_train=Y.loc[0:n]
Y_test1=Y.loc[n:]

X=X_train
Y=Y_train
print(X.shape)
print(X_test1.shape)
print(Y.shape)
print(Y_test1.shape)

(1972, 79)
(493, 79)
(1972, 1)
(493, 1)


In [88]:
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
# X=np.loadtxt('newdata.txt')
# Y=np.loadtxt('newlabels.txt')
# Y=Y.astype(int)


skf = StratifiedKFold(n_splits=5, shuffle=True)
for train_index, test_index in skf.split(X, Y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = Y[train_index], Y[test_index]

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train, y_train) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    clf = SVC(kernel='rbf',gamma='scale',decision_function_shape='ovo')
    clf.fit(X_under, y_under)
    predict_class = clf.predict(X_test)
    
       #f1 values
    f1_value=metrics.f1_score(y_test,predict_class,average='macro')
    f1_class_value=metrics.f1_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,y_test,average='macro')
    precision_class_value=metrics.precision_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,y_test,average='macro')
    recall_class_value=metrics.recall_score(predict_class,y_test,average=None).reshape(1, -1)
    
#     scores = model.evaluate(X_train[test], newY_train[test], verbose=0)
#     accuracy.append(scores[1])
    
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
#     print(f1_class)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)
#     f1.append(metrics.f1_score(predictions,y_test,average='macro'))
#     prs.append(metrics.precision_score(predictions,y_test,average='macro'))
#     recall.append
#     print(clf)
#     print(y_test)
#     print(clf.predict(X_test))
#     print(clf.decision_function(X_test))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being

In [89]:
print("For training:")
print("F1 is :"+ str(np.mean(f1)))
print("Precision is: "+str(np.mean(precision)))
print("Recall is: "+str(np.mean(recall)))
print("\n")
print("Average F1 for all the classes is:" +str(average_f1))
print("Average F1 for all the classes is:" +str(average_precision))
print("Average F1 for all the classes is:" +str(average_recall))

For training:
F1 is :0.6627393588798733
Precision is: 0.6679369035401874
Recall is: 0.6810361051399904


Average F1 for all the classes is:[0.98102939 0.26666667 0.70597403 0.95943705 0.71273504 0.93333333
 0.08      ]
Average F1 for all the classes is:[0.97994429 0.2        0.75       0.97894737 0.76666667 0.9
 0.1       ]
Average F1 for all the classes is:[0.98215135 0.4        0.69428571 0.94129187 0.68285714 1.
 0.06666667]


In [90]:
predict_class = clf.predict(X_test1)

#f1 values
f1_value=metrics.f1_score(Y_test1,predict_class,average='macro')

In [91]:
print("For test set")
print("F1 is :"+ str(np.mean(f1)))

For test set
F1 is :0.6627393588798733


In [ ]:
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
# X=np.loadtxt('newdata.txt')
# Y=np.loadtxt('newlabels.txt')
# Y=Y.astype(int)


skf = StratifiedKFold(n_splits=4, shuffle=True)
for train_index, test_index in skf.split(X, Y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = Y[train_index], Y[test_index]

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train, y_train) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    clf=KNeighborsClassifier(n_neighbors=3)
    clf.fit(X_under, y_under)
    predict_class = clf.predict(X_test)
    
       #f1 values
    f1_value=metrics.f1_score(predict_class,y_test,average='macro')
    f1_class_value=metrics.f1_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,y_test,average='macro')
    precision_class_value=metrics.precision_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,y_test,average='macro')
    recall_class_value=metrics.recall_score(predict_class,y_test,average=None).reshape(1, -1)
    
#     scores = model.evaluate(X_train[test], newY_train[test], verbose=0)
#     accuracy.append(scores[1])
    
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
#     print(f1_class)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)
#     f1.append(metrics.f1_score(predictions,y_test,average='macro'))
#     prs.append(metrics.precision_score(predictions,y_test,average='macro'))
#     recall.append
#     print(clf)
#     print(y_test)
#     print(clf.predict(X_test))
#     print(clf.decision_function(X_test))

In [22]:
print("For training:")
print("F1 is :"+ str(np.mean(f1)))
print("Precision is: "+str(np.mean(precision)))
print("Recall is: "+str(np.mean(recall)))
print("\n")
print("Average F1 for all the classes is:" +str(average_f1))
print("Average F1 for all the classes is:" +str(average_precision))
print("Average F1 for all the classes is:" +str(average_recall))

For training:
Accuracy is :nan
F1 is :0.4660067549005767
Precision is: 0.7004464285714287
Recall is: 0.41328215715097316


Average F1 for all the classes is:[0.88011559 0.15515873 0.37209402 0.84349375 0.3842434  0.5952381
 0.0317037 ]
Average F1 for all the classes is:[0.79241071 0.36666667 0.76666667 0.99166667 0.88571429 0.83333333
 0.26666667]
Average F1 for all the classes is:[0.99004071 0.1021645  0.24856897 0.73938629 0.2492876  0.54666667
 0.01686036]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
# X=np.loadtxt('newdata.txt')
# Y=np.loadtxt('newlabels.txt')
# Y=Y.astype(int)


skf = StratifiedKFold(n_splits=4, shuffle=True)
for train_index, test_index in skf.split(X, Y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = Y[train_index], Y[test_index]

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train, y_train) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    clf=GaussianNB()
    clf.fit(X_under, y_under)
    predict_class = clf.predict(X_test)
    
       #f1 values
    f1_value=metrics.f1_score(predict_class,y_test,average='macro')
    f1_class_value=metrics.f1_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,y_test,average='macro')
    precision_class_value=metrics.precision_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,y_test,average='macro')
    recall_class_value=metrics.recall_score(predict_class,y_test,average=None).reshape(1, -1)
    
#     scores = model.evaluate(X_train[test], newY_train[test], verbose=0)
#     accuracy.append(scores[1])
    
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
#     print(f1_class)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)
#     f1.append(metrics.f1_score(predictions,y_test,average='macro'))
#     prs.append(metrics.precision_score(predictions,y_test,average='macro'))
#     recall.append
#     print(clf)
#     print(y_test)
#     print(clf.predict(X_test))
#     print(clf.decision_function(X_test))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: R

In [6]:
print("For training:")
print("F1 is :"+ str(np.mean(f1)))
print("Precision is: "+str(np.mean(precision)))
print("Recall is: "+str(np.mean(recall)))
print("\n")
print("Average F1 for all the classes is:" +str(average_f1))
print("Average F1 for all the classes is:" +str(average_precision))
print("Average F1 for all the classes is:" +str(average_recall))

For training:
F1 is :0.5753626005156598
Precision is: 0.59442360811251
Recall is: 0.5942910590006342


Average F1 for all the classes is:[0.92108016 0.16666667 0.39930556 0.91080717 0.75660173 0.86666667
 0.00641026]
Average F1 for all the classes is:[0.87465573 0.20833333 0.53333333 0.9375     0.73214286 0.79166667
 0.08333333]
Average F1 for all the classes is:[0.97326668 0.14583333 0.34924242 0.89104021 0.79732143 1.
 0.00333333]


In [8]:
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
# X=np.loadtxt('newdata.txt')
# Y=np.loadtxt('newlabels.txt')
# Y=Y.astype(int)


skf = StratifiedKFold(n_splits=4, shuffle=True)
for train_index, test_index in skf.split(X, Y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = Y[train_index], Y[test_index]

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train, y_train) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    clf=RandomForestClassifier(n_estimators=3, bootstrap = True, max_features = 'sqrt')
    clf.fit(X_under, y_under)
    predict_class = clf.predict(X_test)
    
       #f1 values
    f1_value=metrics.f1_score(predict_class,y_test,average='macro')
    f1_class_value=metrics.f1_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,y_test,average='macro')
    precision_class_value=metrics.precision_score(predict_class,y_test,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,y_test,average='macro')
    recall_class_value=metrics.recall_score(predict_class,y_test,average=None).reshape(1, -1)
    
#     scores = model.evaluate(X_train[test], newY_train[test], verbose=0)
#     accuracy.append(scores[1])
    
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
#     print(f1_class)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)
#     f1.append(metrics.f1_score(predictions,y_test,average='macro'))
#     prs.append(metrics.precision_score(predictions,y_test,average='macro'))
#     recall.append
#     print(clf)
#     print(y_test)
#     print(clf.predict(X_test))
#     print(clf.decision_function(X_test))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validati

In [9]:
print("For training:")
print("F1 is :"+ str(np.mean(f1)))
print("Precision is: "+str(np.mean(precision)))
print("Recall is: "+str(np.mean(recall)))
print("\n")
print("Average F1 for all the classes is:" +str(average_f1))
print("Average F1 for all the classes is:" +str(average_precision))
print("Average F1 for all the classes is:" +str(average_recall))

For training:
F1 is :0.5666378889169036
Precision is: 0.5614874039816372
Recall is: 0.6177808560619076


Average F1 for all the classes is:[0.96960887 0.29166667 0.47424242 0.88680807 0.52747253 0.81666667
 0.        ]
Average F1 for all the classes is:[0.96880469 0.20833333 0.43333333 0.92708333 0.51785714 0.875
 0.        ]
Average F1 for all the classes is:[0.97043414 0.5        0.62777778 0.85125407 0.54166667 0.83333333
 0.        ]
